# Linear-Log Model 

## 0.导入库

In [111]:
import math
import pickle
import re
from collections import Counter

import nltk
import numpy as np
import pandas as pd
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from scipy.sparse import csr_matrix
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

In [2]:
nltk.download("wordnet")
nltk.download("omw-1.4")
nltk.download("averaged_perceptron_tagger")

[nltk_data] Error loading wordnet: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading omw-1.4: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [Errno 11001] getaddrinfo failed>


False

## 1.数据导入与预处理

In [3]:
df = pd.read_csv(
    "ag_news_csv/train.csv", header=None, names=["label", "title", "description"]
)

In [4]:
print(df.head())

   label                                              title  \
0      3  Wall St. Bears Claw Back Into the Black (Reuters)   
1      3  Carlyle Looks Toward Commercial Aerospace (Reu...   
2      3    Oil and Economy Cloud Stocks' Outlook (Reuters)   
3      3  Iraq Halts Oil Exports from Main Southern Pipe...   
4      3  Oil prices soar to all-time record, posing new...   

                                         description  
0  Reuters - Short-sellers, Wall Street's dwindli...  
1  Reuters - Private investment firm Carlyle Grou...  
2  Reuters - Soaring crude prices plus worries\ab...  
3  Reuters - Authorities have halted oil export\f...  
4  AFP - Tearaway world oil prices, toppling reco...  


In [115]:
def replace_space(word):
    return re.sub(r"[-\\/&]", " ", word)

In [6]:
df["title"] = df["title"].apply(replace_space)
df["description"] = df["description"].apply(replace_space)

In [116]:
def replace_num(word):
    return re.sub(r"\d+", "<NUM>", word)

In [8]:
df["title"] = df["title"].apply(replace_num)
df["description"] = df["description"].apply(replace_num)

In [117]:
def separate_num(word):
    return re.sub(r"(<NUM>)", r" \1 ", word)

In [10]:
df["title"] = df["title"].apply(separate_num)
df["description"] = df["description"].apply(separate_num)

In [11]:
print(df)

        label                                              title  \
0           3  Wall St. Bears Claw Back Into the Black (Reuters)   
1           3  Carlyle Looks Toward Commercial Aerospace (Reu...   
2           3    Oil and Economy Cloud Stocks' Outlook (Reuters)   
3           3  Iraq Halts Oil Exports from Main Southern Pipe...   
4           3  Oil prices soar to all time record, posing new...   
...       ...                                                ...   
119995      1  Pakistan's Musharraf Says Won't Quit as Army C...   
119996      2                  Renteria signing a top shelf deal   
119997      2                    Saban not going to Dolphins yet   
119998      2                                  Today's NFL games   
119999      2                       Nets get Carter from Raptors   

                                              description  
0       Reuters   Short sellers, Wall Street's dwindli...  
1       Reuters   Private investment firm Carlyle Grou...  
2  

In [118]:
def tokenize(text):
    return text.split()

In [13]:
df["tokens"] = df["title"].apply(tokenize) + df["description"].apply(tokenize)

In [14]:
print(df["tokens"])

0         [Wall, St., Bears, Claw, Back, Into, the, Blac...
1         [Carlyle, Looks, Toward, Commercial, Aerospace...
2         [Oil, and, Economy, Cloud, Stocks', Outlook, (...
3         [Iraq, Halts, Oil, Exports, from, Main, Southe...
4         [Oil, prices, soar, to, all, time, record,, po...
                                ...                        
119995    [Pakistan's, Musharraf, Says, Won't, Quit, as,...
119996    [Renteria, signing, a, top, shelf, deal, Red, ...
119997    [Saban, not, going, to, Dolphins, yet, The, Mi...
119998    [Today's, NFL, games, PITTSBURGH, at, NY, GIAN...
119999    [Nets, get, Carter, from, Raptors, INDIANAPOLI...
Name: tokens, Length: 120000, dtype: object


In [15]:
df.drop("description", axis=1, inplace=True)
df.drop("title", axis=1, inplace=True)

In [16]:
print(df)

        label                                             tokens
0           3  [Wall, St., Bears, Claw, Back, Into, the, Blac...
1           3  [Carlyle, Looks, Toward, Commercial, Aerospace...
2           3  [Oil, and, Economy, Cloud, Stocks', Outlook, (...
3           3  [Iraq, Halts, Oil, Exports, from, Main, Southe...
4           3  [Oil, prices, soar, to, all, time, record,, po...
...       ...                                                ...
119995      1  [Pakistan's, Musharraf, Says, Won't, Quit, as,...
119996      2  [Renteria, signing, a, top, shelf, deal, Red, ...
119997      2  [Saban, not, going, to, Dolphins, yet, The, Mi...
119998      2  [Today's, NFL, games, PITTSBURGH, at, NY, GIAN...
119999      2  [Nets, get, Carter, from, Raptors, INDIANAPOLI...

[120000 rows x 2 columns]


In [119]:
def lower(tokens):
    return [word.lower() for word in tokens]

In [18]:
df["tokens"] = df["tokens"].apply(lower)

In [19]:
print(df["tokens"])

0         [wall, st., bears, claw, back, into, the, blac...
1         [carlyle, looks, toward, commercial, aerospace...
2         [oil, and, economy, cloud, stocks', outlook, (...
3         [iraq, halts, oil, exports, from, main, southe...
4         [oil, prices, soar, to, all, time, record,, po...
                                ...                        
119995    [pakistan's, musharraf, says, won't, quit, as,...
119996    [renteria, signing, a, top, shelf, deal, red, ...
119997    [saban, not, going, to, dolphins, yet, the, mi...
119998    [today's, nfl, games, pittsburgh, at, ny, gian...
119999    [nets, get, carter, from, raptors, indianapoli...
Name: tokens, Length: 120000, dtype: object


In [20]:
def remove_word_suffixes(word):
    if word.endswith("'s"):
        word = word[:-2]
    # elif word.endswith("s"):
    #    word = word[:-1]
    else:
        return re.sub(r'[.,:()\'"?;#$!]', "", word)

In [21]:
def remove_suffixes(tokens):
    return [remove_word_suffixes(word) for word in tokens]

In [22]:
df["tokens"] = df["tokens"].apply(remove_suffixes)

In [23]:
print(df["tokens"])

0         [wall, st, bears, claw, back, into, the, black...
1         [carlyle, looks, toward, commercial, aerospace...
2         [oil, and, economy, cloud, stocks, outlook, re...
3         [iraq, halts, oil, exports, from, main, southe...
4         [oil, prices, soar, to, all, time, record, pos...
                                ...                        
119995    [None, musharraf, says, wont, quit, as, army, ...
119996    [renteria, signing, a, top, shelf, deal, red, ...
119997    [saban, not, going, to, dolphins, yet, the, mi...
119998    [None, nfl, games, pittsburgh, at, ny, giants,...
119999    [nets, get, carter, from, raptors, indianapoli...
Name: tokens, Length: 120000, dtype: object


In [24]:
def remove_stopwords(tokens):
    return [word for word in tokens if (word not in stopwords) and (word is not None)]

In [25]:
with open("stopwords.txt") as file:
    stopwords = file.read().split(",")

In [26]:
print(stopwords)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'A', 'about', 'above', 'across', 'after', 'again', 'against', 'all', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'am', 'among', 'an', 'and', 'another', 'any', 'anyone', 'anything', 'anywhere', 'are', "aren't", 'around', 'as', 'at', 'b', 'B', 'back', 'be', 'became', 'because', 'become', 'becomes', 'been', 'before', 'behind', 'being', 'below', 'between', 'both', 'but', 'by', 'c', 'C', 'can', 'cannot', "can't", 'could', "couldn't", 'd', 'D', 'did', "didn't", 'do', 'does', "doesn't", 'doing', 'done', "don't", 'down', 'during', 'e', 'E', 'each', 'either', 'enough', 'even', 'ever', 'every', 'everyone', 'everything', 'everywhere', 'f', 'F', 'few', 'find', 'first', 'for', 'four', 'from', 'full', 'further', 'g', 'G', 'get', 'give', 'go', 'h', 'H', 'had', "hadn't", 'has', "hasn't", 'have', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', "here's", 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 

In [27]:
df["tokens"] = df["tokens"].apply(remove_stopwords)

In [28]:
print(df["tokens"])

0         [wall, st, bears, claw, black, reuters, reuter...
1         [carlyle, looks, commercial, aerospace, reuter...
2         [oil, economy, cloud, stocks, outlook, reuters...
3         [iraq, halts, oil, exports, main, southern, pi...
4         [oil, prices, soar, time, record, posing, new,...
                                ...                        
119995    [musharraf, says, wont, quit, army, chief, kar...
119996    [renteria, signing, top, shelf, deal, red, sox...
119997    [saban, going, dolphins, miami, dolphins, cour...
119998    [nfl, games, pittsburgh, ny, giants, time, <nu...
119999    [nets, carter, raptors, indianapolis, star, vi...
Name: tokens, Length: 120000, dtype: object


In [29]:
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith("J"):
        return wordnet.ADJ
    elif treebank_tag.startswith("V"):
        return wordnet.VERB
    elif treebank_tag.startswith("N"):
        return wordnet.NOUN
    elif treebank_tag.startswith("R"):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [30]:
def lemmatize_with_pos(tokens):
    pos_tagged = pos_tag(tokens)
    return [
        lemmatizer.lemmatize(token, get_wordnet_pos(pos)) for token, pos in pos_tagged
    ]

In [31]:
lemmatizer = WordNetLemmatizer()

df["tokens"] = df["tokens"].apply(lemmatize_with_pos)

In [32]:
print(df["tokens"])

0         [wall, st, bear, claw, black, reuters, reuters...
1         [carlyle, look, commercial, aerospace, reuters...
2         [oil, economy, cloud, stock, outlook, reuters,...
3         [iraq, halt, oil, export, main, southern, pipe...
4         [oil, price, soar, time, record, pose, new, me...
                                ...                        
119995    [musharraf, say, wont, quit, army, chief, kara...
119996    [renteria, sign, top, shelf, deal, red, sox, g...
119997    [saban, go, dolphin, miami, dolphin, courtship...
119998    [nfl, game, pittsburgh, ny, giant, time, <num>...
119999    [net, carter, raptor, indianapolis, star, vinc...
Name: tokens, Length: 120000, dtype: object


In [33]:
df.to_pickle("processed_train_data.pkl")

## 2.TF-IDF编码

In [4]:
df = pd.read_pickle("processed_train_data.pkl")

In [5]:
print(df)

        label                                             tokens
0           3  [wall, st, bear, claw, black, reuters, reuters...
1           3  [carlyle, look, commercial, aerospace, reuters...
2           3  [oil, economy, cloud, stock, outlook, reuters,...
3           3  [iraq, halt, oil, export, main, southern, pipe...
4           3  [oil, price, soar, time, record, pose, new, me...
...       ...                                                ...
119995      1  [musharraf, say, wont, quit, army, chief, kara...
119996      2  [renteria, sign, top, shelf, deal, red, sox, g...
119997      2  [saban, go, dolphin, miami, dolphin, courtship...
119998      2  [nfl, game, pittsburgh, ny, giant, time, <num>...
119999      2  [net, carter, raptor, indianapolis, star, vinc...

[120000 rows x 2 columns]


In [6]:
words_counter = Counter()
for tokens in df["tokens"]:
    words_counter.update(tokens)
vocabulary = dict(words_counter)

In [7]:
counter = 0
for key, value in vocabulary.items():
    if counter < 20:
        print(f"{key}: {value}")
        counter += 1
    else:
        break

wall: 1500
st: 1679
bear: 722
claw: 36
black: 836
reuters: 17270
short: 924
seller: 105
dwindle: 48
band: 240
ultra: 81
cynic: 6
see: 1861
green: 864
carlyle: 16
look: 2786
commercial: 541
aerospace: 129
private: 721
investment: 986


In [8]:
def compute_tf(tokens):
    tf = Counter(tokens)
    for i in tf:
        tf[i] = (1 + math.log10(tf[i])) if tf[i] != 0 else 0
    return dict(tf)

In [9]:
TF = [compute_tf(tokens) for tokens in df["tokens"]]

In [10]:
counter = 0
for i in TF:
    if counter < 10:
        print(i)
        counter += 1
    else:
        break

{'wall': 1.3010299956639813, 'st': 1.0, 'bear': 1.0, 'claw': 1.0, 'black': 1.0, 'reuters': 1.3010299956639813, 'short': 1.0, 'seller': 1.0, 'dwindle': 1.0, 'band': 1.0, 'ultra': 1.0, 'cynic': 1.0, 'see': 1.0, 'green': 1.0}
{'carlyle': 1.3010299956639813, 'look': 1.0, 'commercial': 1.0, 'aerospace': 1.0, 'reuters': 1.3010299956639813, 'private': 1.0, 'investment': 1.0, 'firm': 1.0, 'group': 1.0, 'reputation': 1.0, 'make': 1.0, 'time': 1.0, 'occasionally': 1.0, 'controversial': 1.0, 'play': 1.0, 'defense': 1.0, 'industry': 1.0, 'quietly': 1.0, 'place': 1.0, 'bet': 1.0, 'market': 1.0}
{'oil': 1.0, 'economy': 1.3010299956639813, 'cloud': 1.0, 'stock': 1.3010299956639813, 'outlook': 1.3010299956639813, 'reuters': 1.3010299956639813, 'soar': 1.0, 'crude': 1.0, 'price': 1.0, 'plus': 1.0, 'worry': 1.0, 'earnings': 1.0, 'expect': 1.0, 'hang': 1.0, 'market': 1.0, 'week': 1.0, 'depth': 1.0, 'summer': 1.0, 'doldrums': 1.0}
{'iraq': 1.3010299956639813, 'halt': 1.3010299956639813, 'oil': 1.477121254

In [11]:
def compute_idf(dft, df_tokens_len):
    return math.log10(df_tokens_len / dft)

In [12]:
IDF = {word: compute_idf(dft, len(df["tokens"])) for word, dft in vocabulary.items()}

In [13]:
counter = 0
for key, value in IDF.items():
    if counter < 20:
        print(f"{key}: {value}")
        counter += 1
    else:
        break

wall: 1.9030899869919435
st: 1.8541305499095762
bear: 2.2206440484779857
claw: 3.5228787452803374
black: 2.1569749686086084
reuters: 0.8418889084801661
short: 2.113509274827518
seller: 3.057991946977687
dwindle: 3.3979400086720375
band: 2.6989700043360187
ultra: 3.170696227168975
cynic: 4.301029995663981
see: 1.809434872916858
green: 2.1426675035687315
carlyle: 3.8750612633917
look: 1.6342001339596801
commercial: 2.3459839809410554
aerospace: 2.968591535748376
private: 2.2212459813281957
investment: 2.0853043311064137


In [91]:
data = []
indices = []
indptr = [0]

In [92]:
word_list = list(IDF.keys())
word_to_index = {word: i for i, word in enumerate(word_list)}

In [93]:
for i in range(len(TF)):
    for word, tf in TF[i].items():
        if word in IDF:
            tf_idf = tf * IDF[word]
            data.append(tf_idf)
            indices.append(word_to_index[word])
    indptr.append(len(data))

In [94]:
tf_idf = csr_matrix((data, indices, indptr), shape=(len(TF), len(IDF)), dtype=float)

In [95]:
print(tf_idf)

  (0, 0)	2.4759771575242944
  (0, 1)	1.8541305499095762
  (0, 2)	2.2206440484779857
  (0, 3)	3.5228787452803374
  (0, 4)	2.1569749686086084
  (0, 5)	1.0953227229495044
  (0, 6)	2.113509274827518
  (0, 7)	3.057991946977687
  (0, 8)	3.3979400086720375
  (0, 9)	2.6989700043360187
  (0, 10)	3.170696227168975
  (0, 11)	4.301029995663981
  (0, 12)	1.809434872916858
  (0, 13)	2.1426675035687315
  (1, 14)	5.0415709387081655
  (1, 15)	1.6342001339596801
  (1, 16)	2.3459839809410554
  (1, 17)	2.968591535748376
  (1, 5)	1.0953227229495044
  (1, 18)	2.2212459813281957
  (1, 19)	2.0853043311064137
  (1, 20)	1.6824589675438515
  (1, 21)	1.3535962737769305
  (1, 22)	2.9822712330395684
  (1, 23)	1.4116349065361085
  :	:
  (119998, 7668)	2.8916605252111616
  (119998, 752)	1.7262273343375372
  (119998, 245)	1.432875565762866
  (119998, 86)	0.9259458273953294
  (119998, 1075)	2.1337126609158052
  (119999, 134)	2.466267213451721
  (119999, 4397)	3.483873019361422
  (119999, 9802)	3.5977564948380083
  (119

In [96]:
print(tf_idf.shape)

(120000, 58148)


## 3.构建并训练Log-Linear模型

In [97]:
class LogLinearModel:
    def __init__(self, n_features, n_classes):
        self.n_features = n_features
        self.n_classes = n_classes
        self.weights = np.zeros((n_classes, n_features))

    def train(self, X, y, lr=0.01, epochs=20, batch_size=64):
        n_samples = X.shape[0]
        for epoch in tqdm(range(epochs)):
            batch_losses = []
            shuffled_indices = np.random.permutation(n_samples)
            for start_index in tqdm(range(0, n_samples, batch_size), leave=False):
                end_index = min(start_index + batch_size, n_samples)
                batch_indices = shuffled_indices[start_index:end_index]

                batch_X = X[batch_indices].toarray()
                batch_y = y[batch_indices]

                scores = batch_X.dot(self.weights.T)
                probs = self._softmax(scores)

                loss = self._cross_entropy(probs, batch_y)
                batch_losses.append(loss)

                delta = (probs - batch_y).T.dot(batch_X)
                self.weights -= lr * delta

            epoch_loss = np.mean(batch_losses)
            print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}")

    def predict(self, X):
        scores = X.dot(self.weights.T)
        probs = self._softmax(scores)
        return np.argmax(probs, axis=1)

    def _softmax(self, x):
        exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exp_x / exp_x.sum(axis=1, keepdims=True)

    def _cross_entropy(self, probs, y_true):
        log_probs = -np.log(probs[range(len(probs)), np.argmax(y_true, axis=1)])
        return np.mean(log_probs)

    def save(self, filepath):
        with open(filepath, "wb") as f:
            pickle.dump(self, f)

    @staticmethod
    def load(filepath):
        with open(filepath, "rb") as f:
            return pickle.load(f)

In [98]:
y = np.eye(4)[df["label"] - 1]

In [99]:
print(y)

[[0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 ...
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]]


In [100]:
X_train, X_val, y_train, y_val = train_test_split(
    tf_idf, y, test_size=0.2, random_state=42
)

In [101]:
model = LogLinearModel(X_train.shape[1], 4)

In [105]:
model.train(X_train, y_train, epochs=100)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/1500 [00:00<?, ?it/s]

Epoch 1/100, Loss: 0.0375


  0%|          | 0/1500 [00:00<?, ?it/s]

Epoch 2/100, Loss: 0.0372


...

Epoch 98/100, Loss: 0.0208


  0%|          | 0/1500 [00:00<?, ?it/s]

Epoch 99/100, Loss: 0.0208


  0%|          | 0/1500 [00:00<?, ?it/s]

Epoch 100/100, Loss: 0.0207


In [106]:
model.save("model.pkl")

## 4.评估

In [107]:
loaded_model = LogLinearModel.load("model.pkl")

In [108]:
val_predictions = loaded_model.predict(X_val)
val_accuracy = np.mean(val_predictions == np.argmax(y_val, axis=1))
print(f"Validation Accuracy: {val_accuracy:.4f}")

Validation Accuracy: 0.8732


In [112]:
f1 = f1_score(np.argmax(y_val, axis=1), val_predictions, average="weighted")
print(f"F1 Score: {f1:.4f}")

F1 Score: 0.8733


In [113]:
print("Classification Report:")
print(classification_report(np.argmax(y_val, axis=1), val_predictions))

Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.87      0.87      5956
           1       0.95      0.95      0.95      6058
           2       0.82      0.84      0.83      5911
           3       0.86      0.84      0.85      6075

    accuracy                           0.87     24000
   macro avg       0.87      0.87      0.87     24000
weighted avg       0.87      0.87      0.87     24000



## 5.测试

In [114]:
df = pd.read_csv(
    "ag_news_csv/test.csv", header=None, names=["label", "title", "description"]
)
df["title"] = df["title"].apply(replace_space)
df["description"] = df["description"].apply(replace_space)
df["title"] = df["title"].apply(replace_num)
df["description"] = df["description"].apply(replace_num)
df["title"] = df["title"].apply(separate_num)
df["description"] = df["description"].apply(separate_num)
df["tokens"] = df["title"].apply(tokenize) + df["description"].apply(tokenize)
df.drop("description", axis=1, inplace=True)
df.drop("title", axis=1, inplace=True)
df["tokens"] = df["tokens"].apply(lower)